## Pandas 학습

### 데이터 통합

#### merge
- 데이터를 합칠떄 컬럼을 옆으로 늘려서 합침
- DataBase의 JOIN과 동일한 기능

In [15]:
import pandas as pd

In [25]:
# header 엑셀에서 필요없는 제목줄 등 없애기
# skipfooter 아래쪽에 필요없는 행들을 삭제
df_left = pd.read_excel('../day01/data/sample_1.xlsx', header=1, skipfooter=2, usecols='A:C')
df_left

,국적코드,성별,입국객수
0,A01,남성,106320
1,A01,여성,191436
2,A31,남성,319
3,A31,여성,42
4,A18,남성,158912
5,A18,여성,232943


In [27]:
df_right = pd.read_excel('../day01/data/sample_codemaster.xlsx')
df_right

,국적코드,국적명
0,A01,일본
1,A02,대만
2,A03,홍콩
3,A18,중국
4,A19,이란
5,A22,우즈베키스탄
6,A23,카자흐스탄
7,A99,아시아 기타


In [28]:
## 병합
df_total = pd.merge(left = df_left, right=df_right, how='left', left_on='국적코드',right_on='국적코드')
df_total

,국적코드,성별,입국객수,국적명
0,A01,남성,106320,일본
1,A01,여성,191436,일본
2,A31,남성,319,NaN
3,A31,여성,42,NaN
4,A18,남성,158912,중국
5,A18,여성,232943,중국


In [29]:
## DB INNER JOIN과 동일
df_total = pd.merge(left = df_left, right=df_right, how='inner', left_on='국적코드',right_on='국적코드')
df_total    # 일치하지 않는 값을 생략(삭제)

,국적코드,성별,입국객수,국적명
0,A01,남성,106320,일본
1,A01,여성,191436,일본
2,A18,남성,158912,중국
3,A18,여성,232943,중국


#### append
- 데이터를 합칠때 데이터 수가 늘도록 합치는 방법
- DB에 UNION 이라는 기능과 완전 동일

In [30]:
## append를 하기위한 데이터하나 더 생성
df_letf2 = pd.read_excel('./data/sample_2.xlsx', header=1, skipfooter=2, usecols='A:D')
df_letf2

,국적코드,성별,입국객수,전년동기
0,A01,남성,92556,75992
1,A01,여성,163737,115397
2,A18,남성,155540,129908
3,A18,여성,249023,191236


In [ ]:
## df_all = df_total + df_total2
## pandas 2 버전 이상에서는 append() 함수가 없어짐
## pd.concat()으로 변경
df_all = df_left.append(df_letf2)
df_all

In [32]:
## 이렇게 합칠시 오류발생
pd.concat([df_left, df_letf2])

,국적코드,성별,입국객수,전년동기
0,A01,남성,106320,NaN
1,A01,여성,191436,NaN
2,A31,남성,319,NaN
3,A31,여성,42,NaN
4,A18,남성,158912,NaN
5,A18,여성,232943,NaN
0,A01,남성,92556,75992.0
1,A01,여성,163737,115397.0
2,A18,남성,155540,129908.0
3,A18,여성,249023,191236.0


In [33]:
## df_left2를 가지고 df_total2를 만듬. 국적명을 제대로 받기 위해서
df_total2 = pd.merge(left=df_letf2, right=df_right, how='left', left_on='국적코드', right_on='국적코드')
df_total2

,국적코드,성별,입국객수,전년동기,국적명
0,A01,남성,92556,75992,일본
1,A01,여성,163737,115397,일본
2,A18,남성,155540,129908,중국
3,A18,여성,249023,191236,중국


In [34]:
# ignore_index는 DF의 자체 인덱스를 무시하고 새로 인덱스를 생성
df_all=pd.concat([df_total, df_total2], ignore_index=True)
df_all

,국적코드,성별,입국객수,국적명,전년동기
0,A01,남성,106320,일본,NaN
1,A01,여성,191436,일본,NaN
2,A18,남성,158912,중국,NaN
3,A18,여성,232943,중국,NaN
4,A01,남성,92556,일본,75992.0
5,A01,여성,163737,일본,115397.0
6,A18,남성,155540,중국,129908.0
7,A18,여성,249023,중국,191236.0


In [35]:
## DF 저장.pd.to_...
# index=False DF가 만드는 임의의 인덱스는 저장안됨
df_all.to_excel('./data/df_all.xlsx', index=False)

In [39]:
## 기준년월 확인

TypeError: 'Index' object is not callable

#### Pivot 테이블
통계를 내면서 테이블을 세로로된 DF를 가로로 바꿔주는 기능
DB에서 바로 피벗작업 힘듦. 파이썬에서 피벗하는게 훨씬 쉬움

In [ ]:
## aggfunc => sum(총합), mean(평균), min, maxm median(중앙값), count(개수), nunique(중복제거개수)
df_all.pivot_table(values='입국객수', index='국적코드', columns='기준년월', aggfunc='mean')